# The Tale of two cities 

#### The Tale of two cities is a project designed to pick out the similarities in the neighborhoods of two cities. This program uses foursquare API to solve this problem. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy  # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
 
import requests
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Current Location

In [2]:
current_address='North York, Toronto'

geolocator= Nominatim(user_agent='twocity')
location= geolocator.geocode(current_address)
Lat_current= location.latitude
Long_current= location.longitude
print('The location of {} is lat- {} long- {}.'.format(current_address,Lat_current,Long_current))

The location of North York, Toronto is lat- 43.7543263 long- -79.44911696639593.


In [3]:
moving_address='scarborough, toronto'

geolocator= Nominatim(user_agent='twocity')
location= geolocator.geocode(moving_address)
Lat_moving= location.latitude
Long_moving= location.longitude
print('The location of {} is lat- {} long- {}.'.format(moving_address,Lat_moving,Long_moving))

The location of scarborough, toronto is lat- 43.773077 long- -79.257774.


In [4]:
CLIENT_ID = 'MRAQXAXXOZQA4UWZVKUQTCV5CBFSN3FJZ3ZPMB2CGEQFSMDT' # your Foursquare ID
CLIENT_SECRET = 'WN0SPVBMLKK4EUEGZNRZUBO1WWJAUEBGLWC2YH4XYHYEYS2Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


#### For Current Address

In [5]:
LIMIT= 200
RADIUS= 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Lat_current, 
    Long_current, 
    RADIUS, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MRAQXAXXOZQA4UWZVKUQTCV5CBFSN3FJZ3ZPMB2CGEQFSMDT&client_secret=WN0SPVBMLKK4EUEGZNRZUBO1WWJAUEBGLWC2YH4XYHYEYS2Z&v=20180605&ll=43.7543263,-79.44911696639593&radius=1000&limit=200'

In [6]:
results_current = requests.get(url).json()
results_current

{'meta': {'code': 200, 'requestId': '5f082cb67b9ef927efa28b5d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bathurst Manor',
  'headerFullLocation': 'Bathurst Manor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 43.763326309000014,
    'lng': -79.4366802242078},
   'sw': {'lat': 43.74532629099999, 'lng': -79.46155370858406}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a888f7647f8767d37b92f00',
       'name': 'Grill Gate',
       'location': {'address': '832 Sheppard West',
        'crossStreet': 'NE corner of Sheppard & Wilmington',
        'lat': 43.753122978404804,
        'lng': -79.45169001817703,
 

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results_current['response']['groups'][0]['items']
    
current_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
current_nearby_venues =current_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
current_nearby_venues['venue.categories'] = current_nearby_venues.apply(get_category_type, axis=1)

# clean columns
current_nearby_venues.columns = [col.split(".")[-1] for col in current_nearby_venues.columns]

current_nearby_venues.head()

c:\users\shasw\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,Wolfie's Deli,Deli / Bodega,43.754875,-79.442438
2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,Best for Bride in Toronto,Bridal Shop,43.755789,-79.437834
4,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507


In [9]:
current_nearby_venues.shape

(29, 4)

#### Moving Address

In [10]:
LIMIT= 200
RADIUS= 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Lat_moving, 
    Long_moving, 
    RADIUS, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MRAQXAXXOZQA4UWZVKUQTCV5CBFSN3FJZ3ZPMB2CGEQFSMDT&client_secret=WN0SPVBMLKK4EUEGZNRZUBO1WWJAUEBGLWC2YH4XYHYEYS2Z&v=20180605&ll=43.773077,-79.257774&radius=1000&limit=200'

In [11]:
results_moving = requests.get(url).json()
results_moving

{'meta': {'code': 200, 'requestId': '5f082eb6461da410d68dcc63'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 66,
  'suggestedBounds': {'ne': {'lat': 43.78207700900001,
    'lng': -79.24533335909429},
   'sw': {'lat': 43.76407699099999, 'lng': -79.2702146409057}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
      

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results_moving['response']['groups'][0]['items']
    
moving_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
moving_nearby_venues =moving_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
moving_nearby_venues['venue.categories'] = moving_nearby_venues.apply(get_category_type, axis=1)

# clean columns
moving_nearby_venues.columns = [col.split(".")[-1] for col in moving_nearby_venues.columns]

moving_nearby_venues= pd.DataFrame(moving_nearby_venues)
moving_nearby_venues.head()

c:\users\shasw\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,American Eagle Store,Clothing Store,43.776012,-79.258334
2,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
3,SEPHORA,Cosmetics Shop,43.775017,-79.258109
4,DAVIDsTEA,Tea Room,43.776320,-79.258688


In [14]:
current_nearby_venues= pd.DataFrame(current_nearby_venues)
current_nearby_venues.head()

,name,categories,lat,lng
0,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,Wolfie's Deli,Deli / Bodega,43.754875,-79.442438
2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,Best for Bride in Toronto,Bridal Shop,43.755789,-79.437834
4,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507


In [15]:
frames=[current_nearby_venues,moving_nearby_venues]

merged= pd.concat(frames)
merged.head()

,name,categories,lat,lng
0,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,Wolfie's Deli,Deli / Bodega,43.754875,-79.442438
2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,Best for Bride in Toronto,Bridal Shop,43.755789,-79.437834
4,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507


In [16]:
merged_dummy=pd.get_dummies(merged['categories'])
merged_dummy.head()

,American Restaurant,Bakery,Bank,Bar,Bridal Shop,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Diner,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Greek Restaurant,Grocery Store,Gym,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Music Store,Park,Pharmacy,Pizza Place,Playground,Plaza,Restaurant,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Supermarket,Sushi Restaurant,Tea Room,Toy / Game Store,Video Game Store,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Clustering

In [17]:
kclusters = 50

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_dummy)

kmeans.labels_[0:10] 

array([29, 17,  2, 41, 31, 47,  7,  2, 37,  1])

In [18]:
merged.insert(0, 'Cluster Labels', kmeans.labels_)
merged.head()

,Cluster Labels,name,categories,lat,lng
0,29,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,17,Wolfie's Deli,Deli / Bodega,43.754875,-79.442438
2,2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,41,Best for Bride in Toronto,Bridal Shop,43.755789,-79.437834
4,31,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507


In [20]:
map_clusters = folium.Map(location=[Lat_moving, Long_moving], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['lat'], merged['lng'], merged['categories'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters